### 第2章 スワップカーブと金利スワップ

**2. QuantLibによるTiborスワップの評価**

- [addfixing](https://quant.stackexchange.com/questions/39421/fixing-rate-in-quantlib)

In [1]:
import QuantLib as ql; import pandas as pd
def jDT(yyyy,mm,dd): return ql.Date(dd,mm,yyyy)

# 0.カーブ用 初期値
tradeDT,          Tp2,          freqSA,          calJP             =\
jDT(2022,8,19),    2,       ql.Semiannual,     ql.Japan()

mFLLW,                        dcA365,             pdFreqSA         =\
ql.ModifiedFollowing,   ql.Actual365Fixed(),   ql.Period(freqSA)

settleDT = calJP.advance(tradeDT, Tp2, ql.Days)
ql.Settings.instance().evaluationDate = tradeDT

# 1.変動レグの指標金利オブジェクト
curveHDL = ql.RelinkableYieldTermStructureHandle()  
tbrIX = ql.Tibor(pdFreqSA, curveHDL)

# 2.RateHelperとイールドカーブオブジェクト
cHelper =[]
cHelper.append( ql.DepositRateHelper(
                  ql.QuoteHandle(ql.SimpleQuote(0.13636/100)), tbrIX ))
for tnr, rt in \
    [('1y', 0.15249), ('18m', 0.18742), ('2y', 0.20541), ('3y', 0.23156), 
     ('4y', 0.25653), ('5y', 0.28528), ('6y', 0.32341), ('7y', 0.36591), 
     ('8y', 0.40906), ('9y', 0.45471), ('10y', 0.50224)                ]:
    cHelper.append(ql.SwapRateHelper(
                ql.QuoteHandle(ql.SimpleQuote(rt/100)), ql.Period(tnr), 
                calJP, freqSA, mFLLW, dcA365, tbrIX))

curveOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, cHelper, dcA365)
#             PiecewiseFlatForwardと同じ
curveHDL.linkTo(curveOBJ) ; curveOBJ.enableExtrapolation()

# 3.スケジュールオブジェクト 
effDT,              matDT,          dgRULEb,                 EoMf       =\
jDT(2022,8,23), jDT(2024,8,23), ql.DateGeneration.Backward,  False

fixSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dgRULEb, EoMf)
fltSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dgRULEb, EoMf)
# 4.スワップオブジェクトの作成とエンジンの設定
payRcv,                ntlAMT,      cpn,      fixedIX,      sprd          = \
ql.VanillaSwap.Payer, 10_000_000,  0.012 ,   0.13636/100,   0.0

swapOBJ = ql.VanillaSwap(payRcv, ntlAMT, fixSCD, cpn,         dcA365,
                                         fltSCD, tbrIX, sprd, dcA365)
swapENG = ql.DiscountingSwapEngine(curveHDL)
swapOBJ.setPricingEngine(swapENG)

# 現在のfixed指標金利の設定
prevPayDT = settleDT if swapOBJ.floatingSchedule().startDate() >= settleDT \
                   else swapOBJ.floatingSchedule().previousDate(settleDT)
fixedDT = calJP.advance(prevPayDT, -ql.Period(str(Tp2)+'d'))
tbrIX.addFixing(fixedDT,fixedIX,True)

# 5. 計算結果
pd.DataFrame([  
    ['固定レグ時価'    ,swapOBJ.legNPV(0)],   ['変動レグ時価',swapOBJ.legNPV(1)],
    ['スワップ時価 NPV',swapOBJ.NPV()],       ['フェアレート',swapOBJ.fairRate()],
    ['フェアスプレッド',swapOBJ.fairSpread()]         ], columns=['計算結果', '']) 

,計算結果,
0,固定レグ時価,-239781.110343
1,変動レグ時価,41044.531563
2,スワップ時価 NPV,-198736.578780
3,フェアレート,0.002054
4,フェアスプレッド,0.009946


- MakeScheduleの入力例

In [2]:
fixSCD2 = ql.MakeSchedule(effDT, matDT, pdFreqSA, 
                calendar=calJP, rule=dgRULEb, endOfMonth=EoMf, 
                convention=mFLLW, terminalDateConvention=mFLLW)
fixSCD2.dates()

(Date(23,8,2022),
 Date(24,2,2023),
 Date(23,8,2023),
 Date(26,2,2024),
 Date(23,8,2024))

In [3]:
pd.DataFrame(curveOBJ.nodes(), columns=['node','DF'])#[0:6]

,node,DF
0,"August 23rd, 2022",1.000000
1,"February 24th, 2023",0.999309
2,"August 23rd, 2023",0.998477
3,"February 26th, 2024",0.997170
4,"August 23rd, 2024",0.995896
5,"August 25th, 2025",0.993059
6,"August 24th, 2026",0.989777
7,"August 23rd, 2027",0.985823
8,"August 23rd, 2028",0.980744
9,"August 23rd, 2029",0.974642


In [4]:
[(dt, vl) for dt, vl in zip(tbrIX.timeSeries().dates(), tbrIX.timeSeries().values())]

[(Date(19,8,2022), 0.0013636000000000002)]

**Fig 3**  
<img src='swpm2Y1.2Aug19.png' width='600'>

**スワップオブジェクトに関して**  
- VanillaSwapコンストラクター
- Swapコンストラクター では fairRate() やfloatingSchedule() が動かないので 当面はVanillaSwapコンストラクタを使用
- スワップを評価する為 カーブオブジェクトをハンドルへ変更
  - curveHDL = RelinkableYieldTermStructureHandle(curveOBJ)  
  - またはcurveHDL = RelinkableYieldTermStructureHandle()と  
    curveHDL.linkTo(curveOBJ)  
  
- PiecewiseLogLinearDiscount と PiecewiseFlatForward は同じDFを計算

- NPV(), fairRate() メソッド >>   メソッド一覧は dir(swapOBJ)を参照
- (Python 3.6からの)フォーマット済み文字列リテラル(f文字列)
  - f"xxxx {値 : 書式指定子} xxxx"   
  - "xxxx {インデックス : 書式指定子} xxxx".format(値) 

In [5]:
#(動かないSwapコンストラクター)

# tbrIX = ql.Tibor(ql.Period('6m'), curveHDL)
# cpn, sprd, ntnl = [0.012], [0.0], [10000000.0]

# effDT, matDT = ql.Date('2022-08-15'), jDT(2024-08-15')
# freq, calJP = ql.Period('6m'), calJP
# mFLLW, tConv = ql.ModifiedFollowing, ql.ModifiedFollowing
# dgRULEb, EoMf = ql.DateGeneration.Backward, False

# fixSCD = ql.MakeSchedule(effDT, matDT, freq, calendar=calJP, convention=mFLLW, rule=dgRULEb)
# #fixSCD =    ql.Schedule(effDT, matDT, freq, calJP, mFLLW, tConv, dgRULEb, EoMf)
# fixedLEG = ql.FixedRateLeg(fixSCD, ql.Actual365Fixed(), ntnl, cpn)

# #leg = ql.IborLeg([100], schedule, index, ql.Actual360(), ql.ModifiedFollowing, fixingDays=[2], 
# #                 gearings=[1], spreads=[0], caps=[0])
# fltSCD = ql.MakeSchedule(effDT, matDT, freq,  calendar=calJP, convention=mFLLW, rule=dgRULEb)
# floatLEG = ql.IborLeg(ntnl, fltSCD, tbrIX, ql.Actual365Fixed(),spreads=sprd)

# swapOBJ = ql.Swap(fixedLEG, floatLEG)


**2.1 RateHelperとイールドカーブオブジェクト**

**Fig 4**  
<img src='TiborCurve08192022.png' width='600'>

In [6]:
# タプル -> データフレーム
display( pd.DataFrame( 
                    ([1, 2, 3],
                     [4, 5, 6]) , columns=['A', 'B', 'C']) )
# 辞書型 -> データフレーム
display( pd.DataFrame(
                ({'A':1, 'B':2,'C':3},
                 {'A':4, 'B':5,'C':6}) ,    index=[0, 1] ) )

,A,B,C
0,1,2,3
1,4,5,6


,A,B,C
0,1,2,3
1,4,5,6


In [7]:
print(list(fixSCD))

[Date(23,8,2022), Date(24,2,2023), Date(23,8,2023), Date(26,2,2024), Date(23,8,2024)]


In [8]:
fixSCDmk = ql.MakeSchedule(effDT, matDT, pdFreqSA)
print(list(fixSCDmk))

[Date(23,8,2022), Date(23,2,2023), Date(23,8,2023), Date(23,2,2024), Date(23,8,2024)]


In [9]:
fixSCDmk = ql.MakeSchedule(effDT, matDT, pdFreqSA, calendar=calJP)
print(list(fixSCDmk))

[Date(23,8,2022), Date(24,2,2023), Date(23,8,2023), Date(26,2,2024), Date(23,8,2024)]


**3. キャッシュフロー表 (as\_couponへのキャスト)**

<img src='as_couponキャストコード.png' width='600'>

- floatingScheduleにeffDTとmatDTは含まれる

In [10]:
help(swapOBJ.floatingSchedule())

Help on Schedule in module QuantLib.QuantLib object:

class Schedule(builtins.object)
 |  Schedule(*args)
 |  
 |  Proxy of C++ Schedule class.
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, i)
 |      __getitem__(Schedule self, Integer i) -> Date
 |  
 |  __init__(self, *args)
 |      __init__(Schedule self, DateVector arg2, Calendar calendar=NullCalendar(), BusinessDayConvention const convention=Unadjusted, ext::optional< BusinessDayConvention > terminationDateConvention=ext::nullopt, ext::optional< Period > tenor=ext::nullopt, ext::optional< DateGeneration::Rule > rule=ext::nullopt, ext::optional< bool > endOfMonth=ext::nullopt, BoolVector isRegular=std::vector< bool >(0)) -> Schedule
 |      __init__(Schedule self, Date effectiveDate, Date terminationDate, Period tenor, Calendar calendar, BusinessDayConvention convention, BusinessDayConvention terminationDateConvention, DateGeneration::Rule rule, bool endOfMonth, Date firstDate=Date(), Date nextToLastDate=Date()) -> Sche

- legオブジェクトの日付によるdiscountメソッド例

In [11]:
# print(swapOBJ.floatingSchedule()[1])
# print(curveOBJ.discount(swapOBJ.floatingSchedule()[1]))

**as_couponへのキャスト**
- as_coupon(), as_fixed_rate_coupon(), as_floating_rate_couponメソッド  
( https://quant.stackexchange.com/questions/66056/in-python-quantlib-how-to-identify-principal-and-interest-cashflows )

In [12]:
list(swapOBJ.leg(1))

[<QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001D39E659B70> >,
 <QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001D39E659AE0> >,
 <QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001D39E659C90> >,
 <QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001D39E6737E0> >]

In [13]:
#list(swapOBJ.leg(1))[0].amount()

In [14]:
ql.as_floating_rate_coupon(swapOBJ.leg(1)[1]).rate()

0.001690818788083828

In [15]:
help(swapOBJ.leg(1)[1])

Help on CashFlow in module QuantLib.QuantLib object:

class CashFlow(Observable)
 |  CashFlow(*args, **kwargs)
 |  
 |  Proxy of C++ CashFlow class.
 |  
 |  Method resolution order:
 |      CashFlow
 |      Observable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      __init__(Observable self) -> Observable
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  amount(self)
 |      amount(CashFlow self) -> Real
 |  
 |  date(self)
 |      date(CashFlow self) -> Date
 |  
 |  hasOccurred(self, *args)
 |      hasOccurred(CashFlow self, Date refDate=Date()) -> bool
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __swig_destroy__ = delete_CashFlow(...)
 |      delete_CashFlow(CashFlow self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  thisown
 |      The membership flag
 |  
 |  ----------------

In [16]:
help(ql.as_floating_rate_coupon(swapOBJ.leg(1)[1]))

Help on FloatingRateCoupon in module QuantLib.QuantLib object:

class FloatingRateCoupon(Coupon)
 |  FloatingRateCoupon(*args, **kwargs)
 |  
 |  Proxy of C++ FloatingRateCoupon class.
 |  
 |  Method resolution order:
 |      FloatingRateCoupon
 |      Coupon
 |      CashFlow
 |      Observable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      __init__(Observable self) -> Observable
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  adjustedFixing(self)
 |      adjustedFixing(FloatingRateCoupon self) -> Rate
 |  
 |  convexityAdjustment(self)
 |      convexityAdjustment(FloatingRateCoupon self) -> Rate
 |  
 |  fixingDate(self)
 |      fixingDate(FloatingRateCoupon self) -> Date
 |  
 |  fixingDays(self)
 |      fixingDays(FloatingRateCoupon self) -> Integer
 |  
 |  gearing(self)
 |      gearing(FloatingRateCoupon self) -> Real
 |  
 |  index(self)
 |      index(FloatingRateCoupon self) -> ext::shared_ptr< InterestRateIndex >
 |  
 |  

- map関数によるas_couponへのキャスト  
**Fig 6**

- データフレーム内包表記？？？

In [17]:
df1 = pd.DataFrame([{'A':1, 'B':1*2}, {'A':2, 'B':2*2}, {'A':3, 'B':3*2}])
df2 = pd.DataFrame({'A':x,  'B':x*2} for x in range(1,4))
print('df1 \n', df1) ; print('df2 \n', df2)

df1 
    A  B
0  1  2
1  2  4
2  3  6
df2 
    A  B
0  1  2
1  2  4
2  3  6


- 変動レグ

In [18]:
dfSWP = pd.DataFrame({
    'fixingDate': cpn.fixingDate().ISO(),
    'accruStart': cpn.accrualStartDate().ISO(),
    'accruEnd':   cpn.accrualEndDate().ISO(),
    'payDate':    cpn.date(),                             # Date class(No ISO form)
    'days':       dcA365.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
    'rate':       cpn.rate(),
    'spread':     cpn.spread(),
    'amount':     cpn.amount(),
    } for cpn in map(ql.as_floating_rate_coupon, swapOBJ.leg(1)))       #変動レグ=1
# ディスカウントファクター(DF)
psDF = [1.0                   for dt in dfSWP.payDate if dt <= settleDT] #past   DF
fuDF = [curveOBJ.discount(dt) for dt in dfSWP.payDate if settleDT < dt ] #future DF
dfSWP = pd.concat([dfSWP, pd.DataFrame(psDF+fuDF, columns=['DF']) ], axis=1)
dfSWP.payDate = dfSWP.payDate.map(lambda x: x.ISO())                         # ISOへ

dfSWP.style.format({'nominal':'{:,.2f}', 'rate':'{:.6%}', 'spread':'{:.3%}',
                  'amount':'{:,.2f}','DF':'{:.8f}'})

,fixingDate,accruStart,accruEnd,payDate,days,rate,spread,amount,DF
0,2022-08-19,2022-08-23,2023-02-24,2023-02-24,185,0.136360%,0.000%,"6,911.40",0.99930934
1,2023-02-21,2023-02-24,2023-08-23,2023-08-23,180,0.169082%,0.000%,"8,338.28",0.99847678
2,2023-08-21,2023-08-23,2024-02-26,2024-02-26,187,0.255717%,0.000%,"13,101.12",0.99717037
3,2024-02-21,2024-02-26,2024-08-23,2024-08-23,179,0.261022%,0.000%,"12,800.82",0.99589555


In [19]:
print('変動レグ時価(手計算): {:,.3f}'.format((dfSWP.amount*dfSWP.DF).sum()) )

変動レグ時価(手計算): 41,044.532


In [20]:
dfSWP.amount.sum()

41151.61766661032

- 固定レグ

In [21]:
import myUtil as mu 
dfFixCF = mu.swapCashFlow(swapOBJ, curveOBJ, leg=0)
display( dfFixCF.style.format(
    {'nominal':'{:,.2f}', 'rate':'{:.6%}', 'amount':'{:,.2f}','DF':'{:.8f}'}) )
print('アニュイティ:', (dfFixCF.days/365 * dfFixCF.DF).sum() )

,nominal,accruStart,accruEnd,payDate,days,rate,amount,DF
0,"10,000,000.00",2022-08-23,2023-02-24,2023-02-24,185,1.200000%,"60,821.92",0.99930934
1,"10,000,000.00",2023-02-24,2023-08-23,2023-08-23,180,1.200000%,"59,178.08",0.99847678
2,"10,000,000.00",2023-08-23,2024-02-26,2024-02-26,187,1.200000%,"61,479.45",0.99717037
3,"10,000,000.00",2024-02-26,2024-08-23,2024-08-23,179,1.200000%,"58,849.32",0.99589555


アニュイティ: 1.9981759195233613


<img src='swpm2Y1.2FloatLegAug19.png' width='800'>

**7. フォワードスワップレート**

**Fig 7**  
<img src='QL-1yx2y-main.png' width='500'>

- ScheduleとNumpyによるAnnuity計算

In [22]:
import numpy as np ; import myUtil as mu ; from myABBR import * 
# フォワードスワップのスケジュールオブジェクト
effDTf, matDTf = mu.jDT(2023,8,23), mu.jDT(2025,8,23) 
fixSCDf = ql.Schedule(effDTf, matDTf, pdFreqSA, calJP, mFLLW, mFLLW, dgRULEb, EoMf)

# アニュイティ計算
discFCT = np.array([curveOBJ.discount(d) for d in fixSCDf][1:]) 
tnrLST = np.diff([dcA365.yearFraction(fixSCDf.startDate(), d) for d in fixSCDf]) 
annuity = np.sum(tnrLST * discFCT)

print('discount factors: ', discFCT)
print('tenor list      : ', tnrLST)
print('アニュイティ     : ', annuity)

discount factors:  [0.99717037 0.99589555 0.99445706 0.99305925]
tenor list      :  [0.51232877 0.49041096 0.50958904 0.49589041]
アニュイティ     :  1.998490138388887


In [23]:
import myUtil as mu ; from myABBR import * 
mu.calcAnnuity(fixSCDf, curveOBJ)

1.998490138388887

- effDT、matDTの修正とswapOBJの再計算

In [24]:
# 3.スケジュールオブジェクト(フォワードスワップ) 
effDTf, matDTf = jDT(2023,8,23), jDT(2025,8,23) 
fixSCDf = ql.Schedule(effDTf, matDTf, pdFreqSA, calJP, mFLLW, mFLLW, dgRULEb, EoMf)
fltSCDf = ql.Schedule(effDTf, matDTf, pdFreqSA, calJP, mFLLW, mFLLW, dgRULEb, EoMf)

# 4.スワップオブジェクトの作成とエンジンの設定
swapOBJf = ql.VanillaSwap(ql.VanillaSwap.Payer, ntlAMT, fixSCDf,        cpn, dcA365,
                                                       fltSCDf, tbrIX, sprd, dcA365)
swapENGf = ql.DiscountingSwapEngine(curveHDL)
swapOBJf.setPricingEngine(swapENGf)

# 5. 計算結果
print('Fixed(leg0) NPV : {:,.2f}'.format(swapOBJf.legNPV(0)))
print('Float(leg1) NPV : {:,.2f}'.format(swapOBJf.legNPV(1)))
print('Swap NPV        : {:,.2f}'.format(swapOBJf.NPV()))
print('fairRate        : {: .6%}'.format(swapOBJf.fairRate()))
print('fairSpread      : {: .6%}'.format(swapOBJf.fairSpread()))

Fixed(leg0) NPV : -239,818.82
Float(leg1) NPV : 54,175.33
Swap NPV        : -185,643.49
fairRate        :  0.271081%
fairSpread      :  0.928919%


In [25]:
# 固定ﾚｸﾞ leg(0)
cf0 = pd.DataFrame({
    'nominal':    cf.nominal(),
    'accruStart': cf.accrualStartDate().ISO(),
    'accruEnd':   cf.accrualEndDate().ISO(),
    'payDate':    cf.date().ISO(),
    'days':       cf.accrualEndDate() - cf.accrualStartDate(),
    'rate':       cf.rate(),
    'amount':     cf.amount(),
    'DF':         curveOBJ.discount(cf.date())
    } for cf in map(ql.as_coupon, swapOBJf.leg(0)))

display( cf0.style.format({'nominal':'{:,.2f}', 'days':'{:.0f}', 'rate':'{:.5%}', 
                  'amount':'{:,.2f}', 'DF':'{:.8f}' }) )
print('アニュイティ:', (cf0.days/365 * cf0.DF).sum() )

,nominal,accruStart,accruEnd,payDate,days,rate,amount,DF
0,"10,000,000.00",2023-08-23,2024-02-26,2024-02-26,187,1.20000%,"61,479.45",0.99717037
1,"10,000,000.00",2024-02-26,2024-08-23,2024-08-23,179,1.20000%,"58,849.32",0.99589555
2,"10,000,000.00",2024-08-23,2025-02-25,2025-02-25,186,1.20000%,"61,150.68",0.99445706
3,"10,000,000.00",2025-02-25,2025-08-25,2025-08-25,181,1.20000%,"59,506.85",0.99305925


アニュイティ: 1.998490138388887
